# Default Settings

In [1]:
# 필요한 라이브러리 가져오기
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import os
from matplotlib import pyplot as plt
import wandb
import numpy as np

# 모듈 캐시 제거 및 재로드
# import importlib
# from _01_code._03_real_world_data_to_tensors import p_cryptocurrency_dataset_dataloader as dataloader

# importlib.reload(dataloader)

# 필요한 함수 가져오기
from _01_code._03_real_world_data_to_tensors.p_cryptocurrency_dataset_dataloader import (
    get_cryptocurrency_data,
    CryptoCurrencyDataset,
    add_next_open_column
)

# 주피터 노트북에서는 os.getcwd()로 현재 작업 디렉토리 설정
CURRENT_FILE_PATH = os.getcwd()

# 체크포인트 파일 경로 설정
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")
if not os.path.isdir(CHECKPOINT_FILE_PATH):
    os.makedirs(CHECKPOINT_FILE_PATH)

# Wandb 설정 초기화 (새로운 세션 생성)
wandb.init(
    project="lstm_regression_btc_krw_next_open",
    name=f"BTC_KRW_Regression_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    mode="online"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yuuun03 (yuuun03-korea-university-of-technology-and-education). Use `wandb login --relogin` to force relogin


# Model

In [2]:
# LSTM 모델 정의
class MyModel(nn.Module):
    """
    LSTM 기반 회귀 모델 정의.
    """
    def __init__(self, n_input, n_output):
        """
        모델 초기화 함수.

        Args:
            n_input (int): 입력 특성(feature)의 수
            n_output (int): 출력 특성(feature)의 수
        """
        super().__init__()
        # LSTM 레이어: 순차 데이터를 처리하며, 시계열 데이터의 패턴을 학습.
        self.lstm = nn.LSTM(
            input_size=n_input,
            hidden_size=256,
            num_layers=3,
            dropout=0.3,
            batch_first=True
        )
        # Fully Connected Layer: LSTM의 출력을 최종 예측값으로 변환.
        self.fcn1 = nn.Linear(in_features=256, out_features=128)
        self.fcn2 = nn.Linear(in_features=128, out_features=n_output)

    def forward(self, x):
        """
        모델의 순전파 연산 정의.

        Args:
            x (torch.Tensor): 입력 데이터 (shape: [batch_size, sequence_length, n_input])

        Returns:
            torch.Tensor: 예측값 (shape: [batch_size, n_output])
        """
        x, _ = self.lstm(x)  # LSTM 레이어를 통해 출력값 생성
        x = x[:, -1, :]  # 시퀀스의 마지막 출력값 선택
        x = torch.relu(self.fcn1(x))  # ReLU 활성화 함수 적용
        x = self.fcn2(x)  # 최종 예측값
        return x

In [3]:
# Backpropagation 함수
def backpropagation_step(model, X_batch, y_batch, optimizer, criterion, device):
    """
    Backpropagation 수행 함수

    Args:
        model (nn.Module): LSTM 모델
        X_batch (torch.Tensor): 입력 데이터
        y_batch (torch.Tensor): 타겟 데이터
        optimizer (torch.optim.Optimizer): 옵티마이저
        criterion (torch.nn.Module): 손실 함수
        device (torch.device): 실행 장치

    Returns:
        float: 배치 손실 값
    """
    # 데이터 장치로 이동
    X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(-1)

    # 옵티마이저 초기화
    optimizer.zero_grad()

    # 모델 예측
    output = model(X_batch)

    # 손실 계산
    loss = criterion(output, y_batch)

    # Backpropagation
    loss.backward()

    # 가중치 업데이트
    optimizer.step()

    return loss.item()

# Data Loader

In [4]:
def get_btc_krw_data(sequence_size=10, validation_size=100, test_size=10, is_regression=True):
    """
    데이터셋을 가져와 DataLoader로 변환.

    Args:
        sequence_size (int): 시퀀스 길이
        validation_size (int): 검증 데이터 크기
        test_size (int): 테스트 데이터 크기
        is_regression (bool): 회귀 문제 여부

    Returns:
        tuple: 학습, 검증, 테스트 데이터 로더
    """
    # 데이터 가공 (Next_Open 포함)
    add_next_open_column()  # BTC_KRW_with_next_open.csv 생성

    # 데이터를 로드하고, Next_Open을 피처로 포함.
    X_train, X_validation, X_test, y_train, y_validation, y_test, _, _, _ = get_cryptocurrency_data(
        sequence_size=sequence_size,
        validation_size=validation_size,
        test_size=test_size,
        target_column="Close",  # 타겟 데이터는 여전히 "Close"
        y_normalizer=1.0e7,
        is_regression=is_regression,
        include_next_open=True  # 추가 피처로 Next_Open 포함
    )

    # 데이터셋 정의
    train_dataset = CryptoCurrencyDataset(X=X_train, y=y_train)
    validation_dataset = CryptoCurrencyDataset(X=X_validation, y=y_validation)
    test_dataset = CryptoCurrencyDataset(X=X_test, y=y_test)

    # DataLoader 생성
    train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
    validation_loader = DataLoader(dataset=validation_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset), shuffle=False)

    return train_loader, validation_loader, test_loader

# Train

In [5]:
def train(model, train_loader, validation_loader, device, epochs=10, lr=1e-4):
    """
    모델 학습 함수 (Backpropagation 포함).

    Args:
        model (nn.Module): LSTM 모델
        train_loader (DataLoader): 학습 데이터 로더
        validation_loader (DataLoader): 검증 데이터 로더
        device (torch.device): 실행 장치
        epochs (int): 학습 에폭 수
        lr (float): 학습률
    """
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    criterion = nn.MSELoss()

    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            # Backpropagation Step
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(-1)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        scheduler.step()
        train_loss /= len(train_loader)
        print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}")
        wandb.log({"Train Loss": train_loss, "Epoch": epoch + 1, "Learning Rate": scheduler.get_last_lr()[0]})

        # 검증 단계
        model.eval()
        validation_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in validation_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(-1)
                output = model(X_batch)
                loss = criterion(output, y_batch)
                validation_loss += loss.item()

        validation_loss /= len(validation_loader)
        print(f"[Epoch {epoch+1}] Validation Loss: {validation_loss:.4f}")
        wandb.log({"Validation Loss": validation_loss, "Epoch": epoch + 1})

    torch.save(model.state_dict(), os.path.join(CHECKPOINT_FILE_PATH, "lstm_model_checkpoint.pt"))


# Test

In [6]:
def test(model, test_loader, device):
    """
    테스트 데이터셋에서 모델 성능 평가.

    Args:
        model (nn.Module): 학습된 모델
        test_loader (DataLoader): 테스트 데이터 로더
        device (torch.device): 실행 장치
    """
    model.to(device)
    model.eval()

    predictions, targets = [], []

    print("[TEST DATA]")
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(-1)
            output = model(X_batch)
            predictions.extend(output.cpu().numpy())
            targets.extend(y_batch.cpu().numpy())

    test_loss = np.mean(np.abs(np.array(predictions) - np.array(targets)))
    print(f"Test Loss (Average): {test_loss:.2f}")
    wandb.log({"Test Loss": test_loss})


# Visualization

In [7]:
# 그래프 시각화 함수
def predict_all(model, test_loader, device):
    """
    테스트 데이터셋의 예측 결과 시각화

    Args:
        model (nn.Module): 학습된 모델
        test_loader (DataLoader): 테스트 데이터 로더
        device (torch.device): 실행 장치
    """
    model.to(device)
    model.eval()

    y_normalizer = 100
    X, TARGET_Y, PREDICTION_Y = [], [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(-1)
            predictions = model(X_batch).squeeze(-1)

            X.extend(range(len(y_batch)))
            TARGET_Y.extend(y_batch.cpu().numpy() * y_normalizer)
            PREDICTION_Y.extend(predictions.cpu().numpy() * y_normalizer)

    plt.figure(figsize=(10, 6))
    plt.plot(X, TARGET_Y, label="Actual")
    plt.plot(X, PREDICTION_Y, label="Predicted")
    plt.title("Regression Results")
    plt.legend()
    plt.show()
    wandb.log({"Test Visualization": wandb.Image(plt)})

# Main

In [8]:
# 메인 실행 코드
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 데이터 로드
    train_loader, validation_loader, test_loader = get_btc_krw_data(sequence_size=10)

    # 모델 생성 (n_input=6: 추가 피처 포함)
    model = MyModel(n_input=6, n_output=1)

    # 모델 학습
    train(model, train_loader, validation_loader, device, epochs=50, lr=1e-3)

    # 테스트
    test(model, test_loader, device)

if __name__ == "__main__":
    main()

Processed file saved to: C:\Users\yscho\git\link_dl\_00_data\k_cryptocurrency\BTC_KRW_with_next_open.csv


C:\Users\yscho\git\link_dl\_01_code\_03_real_world_data_to_tensors\p_cryptocurrency_dataset_dataloader.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)  # 입력 데이터는 항상 float32로 저장
C:\Users\yscho\git\link_dl\_01_code\_03_real_world_data_to_tensors\p_cryptocurrency_dataset_dataloader.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y, dtype=torch.float32)  # 회귀 문제일 경우 float32로 저장


[Epoch 1] Train Loss: 0.4798
[Epoch 1] Validation Loss: 0.0308
[Epoch 2] Train Loss: 0.0396
[Epoch 2] Validation Loss: 0.0300
[Epoch 3] Train Loss: 0.0367
[Epoch 3] Validation Loss: 0.0114
[Epoch 4] Train Loss: 0.0310
[Epoch 4] Validation Loss: 0.0054
[Epoch 5] Train Loss: 0.0339
[Epoch 5] Validation Loss: 0.0081
[Epoch 6] Train Loss: 0.0258
[Epoch 6] Validation Loss: 0.0971
[Epoch 7] Train Loss: 0.0240
[Epoch 7] Validation Loss: 0.0146
[Epoch 8] Train Loss: 0.0219
[Epoch 8] Validation Loss: 0.0047
[Epoch 9] Train Loss: 0.0193
[Epoch 9] Validation Loss: 0.0076
[Epoch 10] Train Loss: 0.0180
[Epoch 10] Validation Loss: 0.0048
[Epoch 11] Train Loss: 0.0177
[Epoch 11] Validation Loss: 0.0134
[Epoch 12] Train Loss: 0.0165
[Epoch 12] Validation Loss: 0.0479
[Epoch 13] Train Loss: 0.0193
[Epoch 13] Validation Loss: 0.0072
[Epoch 14] Train Loss: 0.0175
[Epoch 14] Validation Loss: 0.0100
[Epoch 15] Train Loss: 0.0159
[Epoch 15] Validation Loss: 0.0139
[Epoch 16] Train Loss: 0.0157
[Epoch 16] Va

# Next_Open 추가 시 Test Loss가 13.04에서 0.12로 떨어진 이유

시계열 데이터에서 next_open은 목표 값인 Close와 강한 상관관계를 가지며, 이를 통해 모델이 데이터의 패턴과 의존성을 더 효과적으로 학습할 수 있다. 이 피처가 없으면 모델이 학습해야 할 데이터의 불확실성이 증가해 손실 값이 커지고 예측 성능이 저하된다. 반면 next_open이 포함되면 모델이 더 구체적이고 직관적인 추론이 가능해져 손실 값이 크게 감소하며, 예측 정확도가 크게 향상된다. 따라서 금융 데이터 예측에서 이와 같은 추가적인 피처는 데이터의 가치를 증대시키는 핵심적인 요소로 작용하기에 next_open을 입력 feature로 추가했을 때의 Loss가 눈에 띄게 감소한 것이다.